In [ ]:
# !conda create -n sunode-env 
# !conda activate sunode-env 
# !conda config --add channels conda-forge 
# !conda config --set channel_priority strict 

# !conda install sunode -y
# !conda install lapack scipy -y


Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/homebrew/anaconda3/envs/sunode-env





Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate sunode-env
#
# To deactivate an active environment, use
#
#     $ conda deactivate

usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: unrecognized arguments: -y
usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: unrecognized arguments: -y
usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: unrecognized arguments: -y
Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/homebrew/anaconda3

  added / updated specs:
 

In [1]:
1

1

In [ ]:
def lotka_volterra(t, y, p):
    """Right hand side of Lotka-Volterra equation.

    All inputs are dataclasses of sympy variables, or in the case
    of non-scalar variables numpy arrays of sympy variables.
    """
    return {
        'hares': p.alpha * y.hares - p.beta * y.lynx * y.hares,
        'lynx': p.delta * y.hares * y.lynx - p.gamma * y.lynx,

    }


from sunode.symode import SympyProblem
problem = SympyProblem(
    params={
        # We need to specify the shape of each parameter.
        # Any empty tuple corresponds to a scalar value.
        'alpha': (),
        'beta': (),
        'gamma': (),
        'delta': (),
    },
    states={
        # The same for all state variables
        'hares': (),
        'lynx': (),
    },
    rhs_sympy=lotka_volterra,
    derivative_params=[
        # We need to specify with respect to which variables
        # gradients should be computed.
        ('alpha',),
        ('beta',),
    ],
)

# The solver generates uses numba and sympy to generate optimized C functions
# for the right-hand-side and if necessary for the jacobian, adoint and
# quadrature functions for gradients.
from sunode.solver import Solver
solver = Solver(problem, sens_mode=None, solver='BDF')


import numpy as np
tvals = np.linspace(0, 10)
# We can use numpy structured arrays as input, so that we don't need
# to think about how the different variables are stored in the array.
# This does not introduce any runtime overhead during solving.
y0 = np.zeros((), dtype=problem.state_dtype)
y0['hares'] = 1
y0['lynx'] = 0.1

# We can also specify the parameters by name:
solver.set_params_dict({
    'alpha': 0.1,
    'beta': 0.2,
    'gamma': 0.3,
    'delta': 0.4,
})

output = solver.make_output_buffers(tvals)
solver.solve(t0=0, tvals=tvals, y0=y0, y_out=output)

# We can convert the solution to an xarray Dataset
solver.as_xarray(tvals, output)
# ?.soluti?on_hares.plot()

# Or we can convert it to a numpy record array
# from matplotlib import pyplot as plt
# plt.plot(output.view(problem.state_dtype)['hares'])

<xarray.Dataset> Size: 1kB
Dimensions:           (time: 50)
Coordinates:
  * time              (time) float64 400B 0.0 0.2041 0.4082 ... 9.592 9.796 10.0
Data variables:
    solution_hares    (time) float64 400B 1.0 1.016 1.033 ... 1.38 1.353 1.325
    solution_lynx     (time) float64 400B 0.1 0.1021 0.1044 ... 0.9967 1.046
    parameters_alpha  float64 8B 0.1
    parameters_beta   float64 8B 0.2
    parameters_gamma  float64 8B 0.3
    parameters_delta  float64 8B 0.4

In [ ]:
1

# create a simple pivo-like model


In [6]:
# !pip install sunode sympy numba

import sympy as sp
from sunode.symode import SympyProblem
from sunode.solver import Solver
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt

def model(t, y, p):
    epsilon = 1e-9
    delivered_value_per_customer = ((1 - p.f_) * p.E_ * y.Q) / (y.C + epsilon)
    churn = p.d * max(0, p.T - delivered_value_per_customer) * y.C
    C_dot = p.a * y.M - churn
    Q_dot = p.q0 * p.f_ * p.E_ * y.C
    M_dot = -p.m_decay * y.M
    B_dot = p.p_ * y.C - p.cost_ * p.E_
    return {
        'Q': Q_dot,
        'C': C_dot,
        'M': M_dot,
        'B': B_dot,
    }

problem = SympyProblem(
    params={
        'a': (),
        'd': (),
        'T': (),
        'f_': (),
        'E_': (),
        'q0': (),
        'm_decay': (),
        'p_': (),
        'cost_': (),
    },
    states={
        'Q': (),
        'C': (),
        'M': (),
        'B': (),
    },
    rhs_sympy=model,
    derivative_params=[]
)

solver = Solver(problem, sens_mode=None, solver='BDF')

tvals = np.linspace(0, 100, 200)
y0 = np.zeros((), dtype=problem.state_dtype)
y0['Q'] = 0.0
y0['C'] = 1.0
y0['M'] = 100.0
y0['B'] = 10000.0

solver.set_params_dict({
    'a': 0.1,
    'd': 0.5,
    'T': 1.0,
    'f_': 0.5,
    'E_': 10.0,
    'q0': 0.01,
    'm_decay': 0.02,
    'p_': 100.0,
    'cost_': 50.0,
})

output = solver.make_output_buffers(tvals)
solver.solve(t0=0, tvals=tvals, y0=y0, y_out=output)

ds = solver.as_xarray(tvals, output)
ds.Q.plot(label='Q')
ds.C.plot(label='C')
ds.M.plot(label='M')
ds.B.plot(label='B')
plt.legend()
plt.show()

TypeError: cannot determine truth value of Relational